# 1

In [11]:
from helper import aoc
import numpy as np
year=2021
day=1
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )
prev=None
increased=0

values =list(map(lambda x:int(x.strip()), f.readlines()))
for l in values:
    #l=int(l.strip())
    if prev!=None and l>prev:
        increased += 1

    prev=l
print(increased)
lastThree=[]
increased=0
for l in values:
    if len(lastThree)>=3:
        prev=sum(lastThree)
        lastThree.append(l)
        lastThree.pop(0)
        if sum(lastThree)>prev:
            increased+=1
    else:
        lastThree.append(l)
print(increased)


1121
1065


# 2

In [16]:
from helper import aoc
year=2021
day=2
fileName=aoc.init(day,year)
f = open( fileName  , 'r' )


        

values =list(map(lambda x:tuple(x.strip().split(' ')), f.readlines()))
h=0
d=0

for c,v in values:
    if c=='forward':
        h+=int(v)
    if c=='down':
        d+=int(v)
    if c=='up':
        d-=int(v)
print(h*d)
a=0
h=0
d=0
for c,v in values:
    if c=='forward':
        h+=int(v)
        d+=a*int(v)
    if c=='down':
        a+=int(v)
    if c=='up':
        a-=int(v)
print(h*d)

1840243
1727785422


# 3

In [63]:
from helper import aoc
year=2021
day=3
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )
values=np.array(list(map(lambda x:list(map(int,list(x.strip()))), f.readlines())))
# with np.printoptions(threshold=np.inf):
#     print(values[:,0])
#     print(values.T[0,:])

maxOcc=''
minOcc=''
for v in values.T:
    maxOcc+=str(np.bincount(v).argmax())
    minOcc+=str(np.bincount(v).argmin())
print(int(maxOcc,2)*int(minOcc,2))

def recursiveFilterByOcc(data,i,tiebreaker,func):
    maxOcc=[]
    occurences=np.bincount(data[i,:])
    
    # print(occurences)
    # if(occurences[0]==1):
    # print('data:{}'.format(data[i,:]))
    if(not sum(occurences)==1):
        if(len(occurences)==2 and occurences[0]==occurences[1]):
            choice=tiebreaker
        else:
            choice=func(occurences)
    else:
        choice=data[i,0]

    #print(choice)
    maxOcc.append(choice)
    if(data.shape[0]<=(i+1)): return maxOcc
    fltr=np.asarray(maxOcc)
    filteredData=data.T[np.in1d(data.T[:,i],fltr)].T
    return maxOcc + recursiveFilterByOcc(filteredData,i+1,tiebreaker,func)

#print(maxOcc)
print(int(''.join(map(str,recursiveFilterByOcc(values.T,0,1,np.argmax))),2)*int(''.join(map(str,recursiveFilterByOcc(values.T,0,0,np.argmin))),2))


1025636
793873


# 4

In [64]:
from helper import aoc
year=2021
day=4
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

numbersChosen=f.readline().strip().split(',')

bingoRows=[]
bingoSets=[]
for l in f.readlines():
    l=l.strip()
    if l=='':
        if(len(bingoRows)!=0):
            bingoSets.append(bingoRows)
            bingoRows=[]
        continue
    
        


# 5

In [1]:
from helper import aoc
year=2021
day=5
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 6

In [1]:
from helper import aoc
year=2021
day=6
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 7

In [1]:
from helper import aoc
year=2021
day=7
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 8

In [1]:
from helper import aoc
year=2021
day=8
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 9

In [1]:
from helper import aoc
year=2021
day=9
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 10

In [1]:
from helper import aoc
year=2021
day=10
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 11

In [1]:
from helper import aoc
year=2021
day=11
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 12

In [1]:
from helper import aoc
year=2021
day=12
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 13

In [1]:
from helper import aoc
year=2021
day=13
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 14

In [1]:
from helper import aoc
year=2021
day=14
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 15

In [1]:
from helper import aoc
year=2021
day=15
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 16

In [1]:
from helper import aoc
year=2021
day=16
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 17

In [1]:
from helper import aoc
year=2021
day=17
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 18

In [1]:
from helper import aoc
year=2021
day=18
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 19

In [1]:
from helper import aoc
year=2021
day=19
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 20

In [1]:
from helper import aoc
year=2021
day=20
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 21

In [1]:
from helper import aoc
year=2021
day=21
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 22

In [1]:
from helper import aoc
year=2021
day=22
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 23

In [1]:
from helper import aoc
year=2021
day=23
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )

# 24

In [1]:
from helper import aoc
year=2021
day=24
fileName=aoc.init(day,year)

f = open( fileName  , 'r' )